# OJA Deduplication Challenge

## Imports

### Packages

In [198]:
import os
import warnings

import pandas as pd
import re
from unidecode import unidecode
from tqdm import tqdm

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# from Levenshtein import distance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [199]:
warnings.filterwarnings('ignore')

### Data

In [200]:
# Import from s3
os.system(f"mc cp s3/apalazzolo/Deduplication/wi_dataset.csv wi_dataset.csv")

`s3/apalazzolo/Deduplication/wi_dataset.csv` -> `wi_dataset.csv`
Total: 0 B, Transferred: 203.23 MiB, Speed: 147.70 MiB/s


0

In [201]:
data = pd.read_csv('wi_dataset.csv',
                   lineterminator='\n')

In [202]:
# For now let's work with a smaller extract

# data = original_data.head(10000)

In [203]:
n_ads = len(data)
n_ads

112056

In [204]:
data.head()

,id,title,description,location,country_id,company_name,retrieval_date
0,1,Traineeship Pensioen & Leven in Utrecht,We sturen je door naar je toekomstige opdracht...,Utrecht,NL,NaN,2021-01-19
1,2,DEPOSITARY OFFICER (M/F),DEPOSITARY OFFICER (M/F) DO Recruitment Adviso...,NaN,FR,DO Recruitment Advisors,2021-09-30
2,3,Cautam colegi manipulanti marfa,Descriere Angajam manipulanti marfa din Pitest...,Pitesti,RO,NaN,2021-06-18
3,4,Home Care Assistant,Are you looking for a role as a Care Assistant...,NaN,UK,Cera Care,2021-02-09
4,5,Chef de chantier espaces verts h/f,LE POSTE Vous êtes passionné par les Espaces V...,NaN,FR,ALPHEA CONSEIL,2021-06-08


In [205]:
data.tail()

,id,title,description,location,country_id,company_name,retrieval_date
112051,112052,OPERATORE IMPIANTI AUTOMATICI,OPERATORE IMPIANTI AUTOMATICI SBE VARVIT - Mon...,"Monfalcone, Friuli-Venezia Giulia",IT,S.B.E. VARVIT,2021-01-31
112052,112053,Tarvittaessa töihin kutsuttavia siivoojia,Etsimme useampaa MATALAPAINEPESIJÄÄ tiimiimme ...,NaN,FI,ISS Palvelut Oy,2021-04-01
112053,112054,Timanställning - Lager - Start Augusti,Om Jobbet: Just nu söker vi efter nya medarbet...,Jönköping,SE,Lyreco Sverige AB,2021-08-03
112054,112055,Opérateur de saisie bancaire H/F,Venez nous découvrir sur : https://www.tessi.e...,Fontenay-sous-Bois (94),FR,tessi,2021-05-31
112055,112056,Senior Java developer,Do you want to know what is means to be BOLD? ...,NaN,PT,BOLD International,2021-01-30


## Data preprocessing

In [206]:
# Basic cleaning

data.fillna("", inplace=True)

data[
    ['title', 'description', 'location', 'country_id', 'company_name']
] = data[
    ['title', 'description', 'location', 'country_id', 'company_name']
].apply(lambda x: x.str.replace(r'\W', ' ').apply(lambda x: unidecode(re.sub(' +', ' ', x))).str.strip().str.lower())

In [207]:
data.head()

,id,title,description,location,country_id,company_name,retrieval_date
0,1,traineeship pensioen leven in utrecht,we sturen je door naar je toekomstige opdracht...,utrecht,nl,,2021-01-19
1,2,depositary officer m f,depositary officer m f do recruitment advisors...,,fr,do recruitment advisors,2021-09-30
2,3,cautam colegi manipulanti marfa,descriere angajam manipulanti marfa din pitest...,pitesti,ro,,2021-06-18
3,4,home care assistant,are you looking for a role as a care assistant...,,uk,cera care,2021-02-09
4,5,chef de chantier espaces verts h f,le poste vous etes passionne par les espaces v...,,fr,alphea conseil,2021-06-08


In [208]:
data.tail()

,id,title,description,location,country_id,company_name,retrieval_date
112051,112052,operatore impianti automatici,operatore impianti automatici sbe varvit monfa...,monfalcone friuli venezia giulia,it,s b e varvit,2021-01-31
112052,112053,tarvittaessa toihin kutsuttavia siivoojia,etsimme useampaa matalapainepesijaa tiimiimme ...,,fi,iss palvelut oy,2021-04-01
112053,112054,timanstallning lager start augusti,om jobbet just nu soker vi efter nya medarbeta...,jonkoping,se,lyreco sverige ab,2021-08-03
112054,112055,operateur de saisie bancaire h f,venez nous decouvrir sur https www tessi eu fr...,fontenay sous bois 94,fr,tessi,2021-05-31
112055,112056,senior java developer,do you want to know what is means to be bold b...,,pt,bold international,2021-01-30


## Naive deduplication

In [209]:
duplicates = []

### Add the full duplicates

In [210]:
# Way too long to run
# + sorting the table breaks the code

# for i in tqdm(range(n_ads)):
#     for j in range(i+1,n_ads):
#         if (data.iloc[i, 1] == data.iloc[j, 1]) and (data.iloc[i, 2] == data.iloc[j, 2]):
#             duplicates.append({'id1': i+1, 'id2': j+1, 'type': 'FULL'})

In [211]:
data.sort_values(by=['title', 'description', 'id', 'company_name', 'location'], inplace=True)

In [212]:
# Let's try by also using columns 5 and 3

# i = 0
# j = 1
# while i < n_ads and j < n_ads:
#     if (i%10000 == 0) and (j == i+1):
#         print(i)
#     if data.iloc[i, 1] < data.iloc[j, 1]:
#         i += 1
#         j = i + 1
#     elif data.iloc[i, 1] > data.iloc[j, 1]:
#         j += 1
#     elif data.iloc[i, 2] < data.iloc[j, 2]:
#         i += 1
#         j = i + 1
#     elif data.iloc[i, 2] > data.iloc[j, 2]:
#         j += 1
#     else:
#         duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'FULL'})
#         j += 1

In [213]:
for i in tqdm(range(n_ads-1)):
    j = i+1
    while j < n_ads and data.iloc[j, 1] == data.iloc[i, 1] and data.iloc[j, 2] == data.iloc[i, 2]:
        if data.iloc[j, 5] == data.iloc[i, 5] or len(data.iloc[i, 5]) * len(data.iloc[j, 5]) == 0:
            if data.iloc[j, 3] == data.iloc[i, 3] or len(data.iloc[i, 3]) * len(data.iloc[j, 3]) == 0:
                if len(data.iloc[i, 5]) * len(data.iloc[j, 5]) != 0 or len(data.iloc[i, 3]) * len(data.iloc[j, 3]) != 0:
                    duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'FULL'})
        j += 1

len(duplicates)

100%|██████████| 112055/112055 [01:18<00:00, 1424.53it/s]


278593

In [214]:
len(duplicates)

278593

In [215]:
duplicates[0]

{'id1': 16097, 'id2': 23753, 'type': 'FULL'}

### Add the semantic duplicates

In [216]:
data.sort_values(by=['id'], inplace=True)

In [217]:
final_stopwords_list = stopwords.words(
    'danish') + stopwords.words(
    'dutch') + stopwords.words(
    'english') + stopwords.words(
    'finnish') + stopwords.words(
    'french') + stopwords.words(
    'german') + stopwords.words(
    'hungarian') + stopwords.words(
    'portuguese') + stopwords.words(
    'romanian') + stopwords.words(
    'russian') + stopwords.words(
    'spanish')

In [218]:
data['text'] = data['title'] + ' ' + data['description'] + ' ' + data['location'] + ' ' + data['country_id'] + ' ' + data['company_name']

# Use TF-IDF to vectorize the texts
vectorizer = TfidfVectorizer(stop_words=final_stopwords_list,
                             max_df = 0.01)
tfidf = vectorizer.fit_transform(data['text'])
tfidf

<112056x399438 sparse matrix of type '<class 'numpy.float64'>'
	with 3666825 stored elements in Compressed Sparse Row format>

In [219]:
# Use cosine similarity to compare the text
# But actually way too big output

# similarity_matrix = cosine_similarity(tfidf)

In [220]:
# Identify similar ads pairs
# Needs to be done by steps

# for i in tqdm(range(similarity_matrix.shape[0])):
#     for j in range(i+1, similarity_matrix.shape[1]):
#         if similarity_matrix[i][j] > 0.8 and (
#             (data.iloc[i, 1] != data.iloc[j, 1]) or (data.iloc[i, 2] != data.iloc[j, 2])
#         ):
#             if data.iloc[i, 6] != data.iloc[j, 6]:
#                 duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'TEMPORAL'})
#             else:
#                 if abs(
#                     len(data.iloc[i, 2]) - len(data.iloc[j, 2])
#                 ) / max(
#                     len(data.iloc[i, 2]), len(data.iloc[j, 2])
#                 ) < 0.08:
#                     duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'SEMANTIC'})
#                 else:
#                     duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'PARTIAL'})

In [221]:
def cosine_similarity_by_chunk(start, end):
    if end > n_ads:
        end = n_ads
    return cosine_similarity(X=tfidf[start:end], Y=tfidf)

In [222]:
chunk_size = 10000

for chunk_start in range(0, n_ads, chunk_size):
    similarity_matrix_chunk = cosine_similarity_by_chunk(chunk_start, chunk_start+chunk_size)
    compteur_init = len(duplicates)
    for i in tqdm(range(chunk_size)):
        for j in range(chunk_start+i+1, n_ads):
            if similarity_matrix_chunk[i][j] > 0.95:
                if abs(
                    len(data.iloc[chunk_start+i, 2]) - len(data.iloc[j, 2])
                ) / (1 + min(
                    len(data.iloc[chunk_start+i, 2]), len(data.iloc[j, 2])
                )) > 0.1:
                    duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'PARTIAL'})
                elif data.iloc[chunk_start+i, 6] != data.iloc[j, 6]:
                    duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'TEMPORAL'})
                else:
                    duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'SEMANTIC'})
    compteur_end = len(duplicates)
    print(compteur_end-compteur_init)

100%|██████████| 10000/10000 [05:51<00:00, 28.46it/s]


62928


100%|██████████| 10000/10000 [05:17<00:00, 31.46it/s]


58391


100%|██████████| 10000/10000 [04:46<00:00, 34.92it/s]


...

10294


100%|██████████| 10000/10000 [00:22<00:00, 440.80it/s]


4211


100%|██████████| 10000/10000 [00:00<00:00, 13496.45it/s]

112


## Print duplicates

In [223]:
duplicates = pd.DataFrame(duplicates)
duplicates.sort_values(by=['type'], inplace=True)
duplicates.drop_duplicates(subset=['id1', 'id2'], inplace=True)
duplicates.sort_values(by=['id1', 'id2'], inplace=True)
duplicates

,id1,id2,type
40678,3,86,FULL
40679,3,11168,FULL
40680,3,32721,FULL
40681,3,56648,FULL
40682,3,68245,FULL
...,...,...,...
179181,111533,111583,FULL
655053,111576,111924,TEMPORAL
655054,111622,111643,TEMPORAL
36984,111757,112018,FULL


In [224]:
len(duplicates) - len(duplicates.drop_duplicates(subset=['id1', 'id2']))

0

In [225]:
duplicates[duplicates['id1'] > duplicates['id2']]

,id1,id2,type


In [226]:
duplicates.groupby('type').count()

,id1,id2
type,,
FULL,278593,278593
PARTIAL,5839,5839
SEMANTIC,17706,17706
TEMPORAL,74861,74861


In [227]:
duplicates.to_csv('duplicates.csv', index=False, header=False)

In [228]:
os.system(f"mc cp duplicates.csv s3/apalazzolo/Deduplication/duplicates_tf_idf.csv")

`/home/onyxia/work/deduplication/duplicates.csv` -> `s3/apalazzolo/Deduplication/duplicates.csv`
Total: 0 B, Transferred: 6.49 MiB, Speed: 42.56 MiB/s


0